In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.svm import SVC
import spacy
import re
import string

In [4]:
!pip install unidecode

In [0]:
from unidecode import unidecode

In [0]:
#data = pd.read_csv('final.csv')
data = pd.read_csv('/content/drive/My Drive/BERT/CSV_FINAL_EQUALIZADO.csv')
data = data.drop('Unnamed: 0', axis=1)
#data['review_rate'] = data['review_rate'].fillna(' ')
#data = data.fillna(data.mean())
data = data.drop_duplicates(subset=['review_body'])

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data['review_rate'].value_counts()

50    11580
30    10558
40     4847
20     3335
10     2534
Name: review_rate, dtype: int64

In [0]:
def DATA_BINARIZADOR_RESULTADOS(data):
    RESULTADO_BINARIO = []

    for item in data['review_rate']:
        if item < 40:
            RESULTADO_BINARIO.append(0)
        elif item >= 40:
            RESULTADO_BINARIO.append(1)
        else:
            print("Deu feijoada aqui hein")

    data['RESULTADO_BINARIO'] = RESULTADO_BINARIO
    return data

In [7]:

#data = DATA_BINARIZADOR_RESULTADOS(data)
data.head()

,review_rate,review_body,RESULTADO_BINARIO
0,50,Excelente experiência !! Atendimento impecável...,1
1,30,"Local agradável, porém, já fui em melhores, co...",0
2,50,Esta é a “milésima” vez que venho a este resta...,1
3,40,"O local, o atendimento e a comida, sensacionai...",1
4,50,"Comida espetacular, atendimento ímpar! Frank e...",1


In [7]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.snowball import PortugueseStemmer

def clear_text(text):
    #remove pontuacao, palavras com numeros, deixa o texto em caixa baixa e remove o texto entre colchetes
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = unidecode(text)  #PRECISO PQ AS PESSOAS ESCREVEM SEM ACENTO
    stemizador = PortugueseStemmer()
    text = " ".join([word for word in text.split()
                  if word not in stopwords.words('portuguese')])
    text = stemizador.stem(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
clear = lambda x: clear_text(x)
data_clean = pd.DataFrame(data.review_body.apply(clear))
#data_clean['review_rate'] = data['review_rate'].values
data_clean['RESULTADO_BINARIO'] = data['RESULTADO_BINARIO'].values


In [9]:
print(data['review_body'][12])
data_clean['review_body'][12]

A melhor pedida para um feriadinho são os drinks sofisticados servidos na Adega. Obrigada Abel, Wellington e Antonio: O atendimento de vocês é de excelência!


'melhor pedida feriadinho sao drinks sofisticados servidos adega obrigada abel wellington antonio atendimento voces excelenc'

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_clean[['review_body']], data_clean['RESULTADO_BINARIO'], test_size=0.20, random_state=2020)

In [0]:
t_vector = TfidfVectorizer()
t_vector.fit(data_clean['review_body'])
train_X = t_vector.transform(X_train['review_body']) #Tf-idf-weighted document-term matrix(sparse matrix)
test_X = t_vector.transform(X_test['review_body'])


In [14]:
data['review_body'][0]

'Excelente experiência !! Atendimento impecável do funcionário Franklin. Comida excepcional e ambiente muito agradável. Recomendo!!'

In [13]:
print(X_train['review_body'][0])

excelente experiencia atendimento impecavel funcionario franklin comida excepcional ambiente agradavel recom


In [0]:
vector = t_vector.transform([X_train['review_body'][0]])


In [21]:
print(vector.toarray())
print(vector.shape)
for number in vector.toarray():
  for item in number:
    if str(item) != '0.0':
      print (item)

[[0. 0. 0. ... 0. 0. 0.]]
(1, 55607)
0.2016458141315441
0.15849677991811387
0.12643118504086856
0.12384743154916483
0.19707475613535141
0.32856442644386097
0.2519049041494963
0.5866154653257565
0.42669108123141963
0.30339649426009513
0.27278776968696017


In [17]:
print(t_vector.vocabulary_)
print(t_vector.idf_)


{'excelente': 21945, 'experiencia': 22420, 'atendimento': 4716, 'impecavel': 28133, 'funcionario': 25231, 'franklin': 24695, 'comida': 11402, 'excepcional': 22106, 'ambiente': 2558, 'agradavel': 1484, 'recom': 44688, 'local': 32079, 'porem': 41308, 'ja': 30216, 'melhores': 34144, 'nada': 36058, 'extraordinario': 22706, 'garcon': 25515, 'saber': 47267, 'dar': 14514, 'informacoes': 29081, 'prat': 41645, 'milesima': 34848, 'vez': 54429, 'venho': 54239, 'restaurante': 46002, 'opiniao': 37724, 'disparado': 17499, 'sao': 47973, 'paul': 39195, 'sensacionais': 48499, 'satisfazendo': 48047, 'expectativas': 22376, 'unico': 53496, 'senao': 48467, 'oferecimento': 37371, 'servico': 48798, 'jerez': 30476, 'assim': 4310, 'chegamos': 9941, 'aviso': 5360, 'cobrado': 10744, 'surpresa': 50963, 'cada': 7893, 'dose': 18119, 'custou': 14388, 'terem': 51859, 'avisado': 5339, 'clientes': 10631, 'custo': 14360, 'caso': 9208, 'questao': 43817, 'optamos': 37758, 'nao': 36147, 'beber': 6164, 'achando': 501, 'cort

In [14]:
#print(t_vector.get_feature_names())
print(train_X.shape)


(26283, 55607)


In [0]:
def validar(model, model_name, y, pred):
    print("Os escores do ", model_name," são:")
    print(' acc = ', accuracy_score(y, pred), ("\n"),
      'prec = ', precision_score(y, pred), ("\n"),
      'recall = ', recall_score(y, pred), ("\n"),
      'f1 = ', f1_score(y, pred))

    fpr, tpr, threshold = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)
    print(' auc = ', roc_auc)
    return True

<h1>Modelos</h1>

<h2> Regressão Logística </h2>

<h2>Treino</h2>

In [49]:
log = LogisticRegression()
log.fit(train_X, y_train)
log_pred = log.predict(train_X)


print(' acc = ', accuracy_score(y_train, log_pred), ("\n"),
      'prec = ', precision_score(y_train, log_pred), ("\n"),
      'recall = ', recall_score(y_train, log_pred), ("\n"),
      'f1 = ', f1_score(y_train, log_pred))

fpr, tpr, threshold = roc_curve(y_train, log_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)


 acc =  0.9137845755811741 
 prec =  0.9154423018059382 
 recall =  0.9115361170374886 
 f1 =  0.9134850335980453
 auc =  0.9137815853824877


<h2>Teste</h2>

In [50]:
log_pred_test = log.predict(test_X)

print(' acc = ', accuracy_score(y_test, log_pred_test), ("\n"),
      'prec = ', precision_score(y_test, log_pred_test), ("\n"),
      'recall = ', recall_score(y_test, log_pred_test), ("\n"),
      'f1 = ', f1_score(y_test, log_pred_test))

fpr, tpr, threshold = roc_curve(y_test, log_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)


 acc =  0.8709481053112159 
 prec =  0.8743519365660263 
 recall =  0.8679987889797154 
 f1 =  0.8711637800060771
 auc =  0.8709638987738234


<h2>Random Forest otimizando default</h2>

In [51]:
def random_grid(x, labels):
    param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]}, 
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2, 3, 4]},
    ]   
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model = random_grid(train_X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


<h2>Treino</h2>

In [52]:
rf_pred = model.predict(train_X)
print(' acc = ', accuracy_score(y_train, rf_pred), ("\n"),
      'prec = ', precision_score(y_train, rf_pred), ("\n"),
      'recall = ', recall_score(y_train, rf_pred), ("\n"),
      'f1 = ', f1_score(y_train, rf_pred))

fpr, tpr, threshold = roc_curve(y_train, rf_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)

 acc =  0.9999239051858616 
 prec =  0.9999238037183785 
 recall =  0.9999238037183785 
 f1 =  0.9999238037183785
 auc =  0.9999239050509211


<h2>Teste</h2>

In [53]:
rf_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, rf_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, rf_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, rf_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, rf_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, rf_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

 acc =  0.80992 
 prec =  0.81639 
 recall =  0.8023 
 f1 =  0.80928
 auc =  0.80996


<h2>Random Forest: grid seach otimizando acurácia</h2>

In [54]:
def random_grid(x, labels):
    param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]}, 
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2, 3, 4]},
    ]   
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model = random_grid(train_X, y_train)

KeyboardInterrupt: ignored

<h2>Treino</h2>

In [55]:
rf_pred = model.predict(train_X)
print(' acc = ', accuracy_score(y_train, rf_pred), ("\n"),
      'prec = ', precision_score(y_train, rf_pred), ("\n"),
      'recall = ', recall_score(y_train, rf_pred), ("\n"),
      'f1 = ', f1_score(y_train, rf_pred))

fpr, tpr, threshold = roc_curve(y_train, rf_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)

 acc =  0.9999239051858616 
 prec =  0.9999238037183785 
 recall =  0.9999238037183785 
 f1 =  0.9999238037183785
 auc =  0.9999239050509211


<h2>Teste</h2>

In [56]:
rf_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, rf_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, rf_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, rf_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, rf_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, rf_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

 acc =  0.80992 
 prec =  0.81639 
 recall =  0.8023 
 f1 =  0.80928
 auc =  0.80996


<h2>Random Forest otimizando AUC </h2>

In [0]:
def random_grid(x, labels):
    param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]}, 
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2, 3, 4]},
    ]   
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model = random_grid(train_X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


<h2>Treino</h2>

In [0]:
rf_pred = model.predict(train_X)
print(' acc = ', accuracy_score(y_train, rf_pred), ("\n"),
      'prec = ', precision_score(y_train, rf_pred), ("\n"),
      'recall = ', recall_score(y_train, rf_pred), ("\n"),
      'f1 = ', f1_score(y_train, rf_pred))

fpr, tpr, threshold = roc_curve(y_train, rf_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)

 acc =  0.999809762964654 
 prec =  0.9997720018239854 
 recall =  0.9998479896632971 
 f1 =  0.999809994299829
 auc =  0.9998097178241825


<h2>Teste</h2>

In [0]:
rf_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, rf_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, rf_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, rf_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, rf_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, rf_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

 acc =  0.81205 
 prec =  0.81747 
 recall =  0.80122 
 f1 =  0.80927
 auc =  0.812


<h2>SVM otimizando default</h2>

In [0]:
def svm_grid(x, labels):
    param_grid = [
    {'kernel': ['rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0, 100.0],
        #'degree': [1, 2, 3, 4]}
        'degree': [3]}
    ]   
    grid_search = GridSearchCV(SVC(), param_grid, cv=5)
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model_svc = svm_grid(train_X, y_train)

<h2>Treino</h2>

In [1]:
svm_pred = model.predict(train_X)
print(' acc = ', round(accuracy_score(y_train, svm_pred), 5), ("\n"),
      'prec = ', round(precision_score(y_train, svm_pred), 5), ("\n"),
      'recall = ', round(recall_score(y_train, svm_pred), 5), ("\n"),
      'f1 = ', round(f1_score(y_train, svm_pred), 5))

fpr, tpr, threshold = roc_curve(y_train, svm_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

NameError: ignored

<h2>Teste</h2>

In [0]:
svm_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, svm_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, svm_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, svm_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, svm_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, svm_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

<h2>SVM otimizando acurácia</h2>

In [0]:
def svm_grid(x, labels):
    param_grid = [
    {'kernel': ['rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0, 100.0],
        #'degree': [1, 2, 3]}
        'degree': [3]}
    ]   
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model_svc = svm_grid(train_X, y_train)

In [0]:
SVC().get_params().keys() #os parametros que dá pra alterar

<h2>Treino</h2>

In [0]:
svm_pred = model.predict(train_X)
print(' acc = ', round(accuracy_score(y_train, svm_pred), 5), ("\n"),
      'prec = ', round(precision_score(y_train, svm_pred), 5), ("\n"),
      'recall = ', round(recall_score(y_train, svm_pred), 5), ("\n"),
      'f1 = ', round(f1_score(y_train, svm_pred), 5))

fpr, tpr, threshold = roc_curve(y_train, svm_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

<h2>Teste</h2>

In [0]:
svm_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, svm_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, svm_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, svm_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, svm_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, svm_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

In [0]:
#refazendo os grid search, optando por otimizar a auc

<h2>SVM otimizando AUC</h2>

In [0]:
def svm_grid(x, labels):
    param_grid = [
    {'kernel': ['rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0, 100.0],
        #'degree': [1, 2, 3, 4]}
        'degree': [3]}
    ]   
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model_svc = svm_grid(train_X, y_train)

<h2>Treino</h2>

In [0]:
svm_pred = model.predict(train_X)
print(' acc = ', round(accuracy_score(y_train, svm_pred), 5), ("\n"),
      'prec = ', round(precision_score(y_train, svm_pred), 5), ("\n"),
      'recall = ', round(recall_score(y_train, svm_pred), 5), ("\n"),
      'f1 = ', round(f1_score(y_train, svm_pred), 5))

fpr, tpr, threshold = roc_curve(y_train, svm_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

<h2>Teste</h2>

In [0]:
svm_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, svm_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, svm_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, svm_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, svm_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, svm_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))